/docs/integration/tools

In [ ]:
## Arxiv--Research Paper Search
## Tools cration

from langchain_community.tools import ArxivQueryRun , WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper , ArxivAPIWrapper

In [ ]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results= 1 , doc_content_chars_max= 250)
wiki = WikipediaQueryRun(api_wrapper= api_wrapper_wiki)
wiki.name

In [ ]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results= 1 , doc_content_chars_max= 250)
arxiv = ArxivQueryRun(api_wrapper= api_wrapper_arxiv)
arxiv.name

In [ ]:
tools = [wiki, arxiv]

In [ ]:
## custom tools [RAG tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq

In [ ]:
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API"] = os.getenv("GROQ_API")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["HF_TOKEN"] = os.getenv("HF_Token")

llm=ChatGroq(groq_api_key=os.getenv("GROQ_API"),model="Llama3-8b-8192")
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)


In [ ]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
data = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(data)
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever()
retriever


we need to convert retreiver to a tool

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever , "langsmith-search" , "search any info about langsmith")
retriever_tool.name

In [ ]:
tools = [wiki , arxiv, retriever_tool]

In [ ]:
tools

In [ ]:
## run the tools with agents and llms 

## tools, llm --> AgentExecutor 

## Prompt Template 
from langchain import hub
prompt= hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
## Agents 

from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm , tools , prompt)
agent


In [ ]:
## Agent Executer 
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

In [ ]:
agent_executor.invoke({"input": "what is langcsmith ?"})

In [ ]:
agent_executor.invoke({"input": "what is machine learning ?"})

In [ ]:
agent_executor.invoke({"input": "what is the paper 1706.03762 about ?"})